In [0]:
%pip install kaggle


In [0]:
dbutils.library.restartPython()


In [0]:
import os
with open('tokens_and_api.txt', 'r') as file:
    exec(file.read())


In [0]:
!kaggle datasets download -d usdot/flight-delays -f flights.csv
!kaggle datasets download -d usdot/flight-delays -f airlines.csv
!kaggle datasets download -d usdot/flight-delays -f airports.csv

In [0]:
%pip install kagglehub
import kagglehub


path = kagglehub.dataset_download("bahraleloom/noaa-weather-data-jfk-airport")

print("Path to dataset files:", path)

In [0]:
import os

data_path = "/root/.cache/kagglehub/datasets/bahraleloom/noaa-weather-data-jfk-airport/versions/1"
os.listdir(data_path)


In [0]:
# Recursively copy the directory to DBFS
dbutils.fs.cp(
    "file:/Workspace/Users/kstod@student.agh.edu.pl/databricks-flight-lakehouse",
    "dbfs:/tmp/databricks-flight-lakehouse",
    recurse=True
)

import zipfile
import os

# Find the zip file in the copied directory
src_dir = "/dbfs/tmp/databricks-flight-lakehouse"
zip_files = [f for f in os.listdir(src_dir) if f.endswith(".zip")]
zip_path = os.path.join(src_dir, zip_files[0])

extract_path = "/dbfs/tmp/flights_extracted"
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

file_list = os.listdir(extract_path)
df_files = spark.createDataFrame(
    [(f,) for f in file_list],
    ["filename"]
)
display(df_files)

In [0]:
%pip install azure-storage-blob

from azure.storage.blob import BlobClient

In [0]:
def upload_to_storage_sas(local_path, container_sas_url, blob_name):
    # Remove trailing slash if present
    if container_sas_url.endswith('/'):
        container_sas_url = container_sas_url[:-1]
    # Insert the blob name before the SAS token
    if '?' in container_sas_url:
        base_url, sas_token = container_sas_url.split('?', 1)
        blob_url = f"{base_url}/{blob_name}?{sas_token}"
    else:
        raise ValueError("SAS URL must contain a '?' separating the base URL and the SAS token.")
    blob_client = BlobClient.from_blob_url(blob_url)
    with open(local_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded {local_path} -> {blob_url}")


In [0]:
upload_to_storage_sas(
    "/dbfs/tmp/flights_extracted/flights.csv",
    sas_url,
    "flights.csv"
)

In [0]:
upload_to_storage_sas(
    "airports.csv",
    sas_url,
    "airports.csv"
)

In [0]:
upload_to_storage_sas(
    "airlines.csv",
    sas_url,
    "airlines.csv"
)

In [0]:
upload_to_storage_sas(
    "/root/.cache/kagglehub/datasets/bahraleloom/noaa-weather-data-jfk-airport/versions/1/noaa-weather-data-jfk-airport/jfk_weather_cleaned.csv",
    sas_url,
    "jfk_weather_cleaned.csv"
)

In [0]:
upload_to_storage_sas(
    "/root/.cache/kagglehub/datasets/bahraleloom/noaa-weather-data-jfk-airport/versions/1/noaa-weather-data-jfk-airport/jfk_weather.csv",
    sas_url,
    "jfk_weather.csv"
)